In [1]:
import requests
from bs4 import BeautifulSoup
import json 
import logging
import os
from urllib.parse import urlencode
from datetime import datetime
import time

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
}

In [3]:
base_url = "https://luatvietnam.vn/tim-van-ban.html?Keywords=&SearchOptions=1&SearchByDate=IssueDate&DateFrom=&DateTo=&DocTypeId=11&OrganId=0&EffectStatusId=4&FieldId=28&LanguageId=0&SignerId=0&SignerName=&pSize=50"

In [4]:
reponse = requests.get(base_url,headers=headers)

In [5]:
soup = BeautifulSoup(reponse.content, 'html.parser')

In [6]:
block_contents = soup.find_all('div', class_='block-content')

In [7]:
articles = block_contents[1].find_all('article', class_='doc-article')


In [11]:
hrefs =[]
for article in articles:
    doc_titles = article.find_all(class_='doc-title')
    
    for doc_title in doc_titles:
        links = doc_title.find_all('a', href=True)
        hrefs.extend([link['href'] for link in links])


In [19]:
new = hrefs[:50]

In [21]:
import re
import json
from datetime import datetime
import logging

# Cấu hình logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('law_parser.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

result = {
    "title": "",
    "id_doc": "",
    "location": "Hà Nội",
    "doc_type": "Giao Thông",
    "date": "",
    "base": [],
    "signs": [],
    "data": {},
    "department": "Quốc hội",
    "doc_type_en": "The law"
}

def parse_law_structure(docitems):
    signs_text = []
    date_found = False
    all_content = []
    
    logging.info(f"Bắt đầu parse với số lượng docitems: {len(docitems)}")
    
    for item in docitems:
        text = item.get_text(strip=True)
        
        # Xử lý phần chữ ký và ngày tháng
        if "Luật này được Quốc hội" in text or "CHỦ TỊCH" in text:
            date_found = True
            signs_text.append(text)
            continue
            
        if date_found:
            signs_text.append(text)
            continue

        # Collect all content
        if text:
            all_content.append(text)
    
    # Save all content to data
    result['data'] = {
        "content": "\n".join(all_content)
    }

    # Xử lý phần chữ ký
    if signs_text:
        full_signs = ' '.join(signs_text)
        
        name_match = re.search(r'Đã ký:?\s*([^.]+)', full_signs)
        if name_match:
            name = name_match.group(1).strip()
            result["signs"] = [name]
            
        date_match = re.search(r'Luật này được.*?thông qua ngày (\d+) tháng (\d+) năm (\d+)', full_signs)
        if date_match:
            ngay, thang, nam = date_match.groups()
            result["date"] = f"{ngay}/{thang}/{nam}"



In [15]:
# for i in range(0,25):
#     herf = documents[i]['href']
#     tem_url = "https://luatvietnam.vn" + herf
#     reponse = requests.get(tem_url, headers=headers)
#     soup = BeautifulSoup(reponse.content, 'html.parser')
#     main = soup.select_one('main.main')
#     if not main:
#         print("Could not find main content section")  
#     main_content = main.select_one('div.main-content')
#     wrapper = main_content.select_one('div.wrapper')
#     content_left = wrapper.select_one('div.content-left')
#     document = content_left.select_one('article.the-document')
#     tab_content = document.select_one('div.tab-content.fix-border')
#     tab_noi_dung = tab_content.select_one('div#chidanthaydoi')

    
#     # Xử lý title và id_doc
#     data_title = tab_content.get('data-title')
#     if data_title:
#         match = re.search(r'\b(\d{2}/\d{4}/QH\d{2})\b', data_title)
#         title_part = data_title.split(', số')[0].strip()
#         result["title"] = title_part
#         if match:
#             document_number = match.group(1)
#             result["id_doc"] = document_number

#     # Parse docitems
#     docitems = tab_noi_dung.find_all("div", class_=lambda x: x and x.startswith("docitem"))
#     parse_law_structure(docitems)

#     # Save to JSON file
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     filename = f"{result['title'].replace(' ', '_')}.json"
#     special = "Điều_chỉnh_luật.json"
#     if "_số" in filename:
#         clean_name = re.sub(r'_số.*\.json$', '.json', filename)
#         filename = clean_name


#     try:
#         with open(filename, 'w', encoding='utf-8') as f:
#             json.dump(result, f, ensure_ascii=False, indent=2)
#         print(f"Đã lưu thành công vào file: {filename}")
#     except Exception as e:
#         print(f"Lỗi khi lưu file: {str(e)}")


In [23]:
for i in range(len(new[:50])):
    tem_url = "https://luatvietnam.vn" + new[i]
    reponse = requests.get(tem_url, headers=headers)
    soup = BeautifulSoup(reponse.content, 'html.parser')
    main = soup.select_one('main.main')
    if not main:
        print("Could not find main content section")  
    main_content = main.select_one('div.main-content')
    wrapper = main_content.select_one('div.wrapper')
    content_left = wrapper.select_one('div.content-left')
    document = content_left.select_one('article.the-document')
    tab_content = document.select_one('div.tab-content.fix-border')
    tab_noi_dung = tab_content.select_one('div#chidanthaydoi')

        
    # Xử lý title và id_doc
    data_title = tab_content.get('data-title')
    if data_title:
        match = re.search(r'\b(\d{2}/\d{4}/QH\d{2})\b', data_title)
        title_part = data_title.split(', số')[0].strip()
        result["title"] = title_part
        if match:
            document_number = match.group(1)
            result["id_doc"] = document_number

        # Parse docitems
        docitems = tab_noi_dung.find_all("div", class_=lambda x: x and x.startswith("docitem"))
        parse_law_structure(docitems)

        # Save to JSON file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"Nghi_dinh_giao_thong_{i}.json"
        special = "Điều_chỉnh_luật.json"
        if "_số" in filename:
            clean_name = re.sub(r'_số.*\.json$', '.json', filename)
            filename = clean_name


        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            print(f"Đã lưu thành công vào file: {filename}")
        except Exception as e:
            print(f"Lỗi khi lưu file: {str(e)}")

2024-11-11 21:39:06,228 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443
2024-11-11 21:39:06,446 - DEBUG - https://luatvietnam.vn:443 "GET /thue/nghi-dinh-130-2024-nd-cp-thu-phi-duong-bo-cao-toc-voi-phuong-tien-luu-thong-tren-tuyen-duong-bo-cao-toc-368350-d1.html HTTP/11" 200 None
2024-11-11 21:39:06,541 - INFO - Bắt đầu parse với số lượng docitems: 111
2024-11-11 21:39:06,543 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443
2024-11-11 21:39:06,720 - DEBUG - https://luatvietnam.vn:443 "GET /doanh-nghiep/nghi-dinh-121-2024-nd-cp-sua-doi-nd-139-2018-nd-cp-quy-dinh-kinh-doanh-dich-vu-kiem-dinh-xe-co-gioi-367481-d1.html HTTP/11" 200 None


Đã lưu thành công vào file: Nghi_dinh_giao_thong_0.json


2024-11-11 21:39:06,770 - INFO - Bắt đầu parse với số lượng docitems: 20
2024-11-11 21:39:06,772 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443
2024-11-11 21:39:06,935 - DEBUG - https://luatvietnam.vn:443 "GET /tai-chinh/nghi-dinh-119-2024-nd-cp-cua-chinh-phu-quy-dinh-ve-thanh-toan-dien-tu-giao-thong-duong-bo-367223-d1.html HTTP/11" 200 None


Đã lưu thành công vào file: Nghi_dinh_giao_thong_1.json


2024-11-11 21:39:07,019 - INFO - Bắt đầu parse với số lượng docitems: 288
2024-11-11 21:39:07,021 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443


...

Đã lưu thành công vào file: Nghi_dinh_giao_thong_47.json


2024-11-11 21:39:23,539 - DEBUG - https://luatvietnam.vn:443 "GET /an-ninh-trat-tu/nghi-dinh-27-2010-nd-cp-chinh-phu-50883-d1.html HTTP/11" 200 None
2024-11-11 21:39:23,598 - INFO - Bắt đầu parse với số lượng docitems: 49
2024-11-11 21:39:23,601 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443


Đã lưu thành công vào file: Nghi_dinh_giao_thong_48.json


2024-11-11 21:39:23,852 - DEBUG - https://luatvietnam.vn:443 "GET /giao-thong/nghi-dinh-109-2009-nd-cp-chinh-phu-48156-d1.html HTTP/11" 200 None
2024-11-11 21:39:24,004 - INFO - Bắt đầu parse với số lượng docitems: 71


Đã lưu thành công vào file: Nghi_dinh_giao_thong_49.json
